In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from lazypredict.Supervised import LazyRegressor
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import make_scorer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [3]:
df = pd.read_csv("data/cleaned_train.csv")
df.head()

,id,Place Code,Promotion Name,Store Kind,Store Sales,Store Cost,Is Recyclable?,Store Area,Grocery Area,Frozen Area,...,Department,Bar For Salad,Florist,Coffee Bar,Ready Food,Video Store,Gross Weight,Net Weight,Package Weight,Min. Person Yearly Income
0,mc_ID_0,H11go_ZA,Dimes Off,Deluxe,8760000.00,4292400.00,yes,2842.23,2037.64,481.98,...,Household,1,1,1,1,1,28.20,26.60,1.60,10000.00
1,mc_ID_1,S04ne_WA,Budget Bargains,Supermarket,6360000.00,1971600.00,no,2814.95,2049.72,457.36,...,Snack Foods,0,0,0,0,0,16.57,14.97,1.60,50000.00
2,mc_ID_2,L05es_CA,Shelf Emptiers,Supermarket,10860000.00,4452600.00,yes,2192.32,1322.21,523.32,...,Periodicals,0,1,0,0,0,28.64,27.18,1.45,30000.00
3,mc_ID_4,M10da_YU,Sale Winners,Deluxe,11560000.00,4970800.00,no,2862.30,1872.19,593.93,...,Produce,1,1,1,1,1,12.62,9.71,2.91,50000.00
4,mc_ID_5,S03le_WA,Weekend Discount,Supermarket,5220000.00,1618200.00,yes,1970.17,1236.07,440.92,...,Household,0,0,1,0,0,15.41,13.95,1.45,30000.00


In [4]:
target = "Cost"
X = df.drop(columns=target)
y = df[target]

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
ord_categorical_columns = ['Place Code', 'Promotion Name', 'Store Kind', 'Is Recyclable?',
                            'Children', 'Degree', 'Work', 'Oreder Brand',
                            'Product', 'Department']

hot_categorical_columns = ['Marriage', 'Gender']

all_categorical_columns = X.select_dtypes("object").columns

numeric_columns = X.select_dtypes("float").columns

In [7]:
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OrdinalEncoder(), ord_categorical_columns),
        ('ohe', OneHotEncoder(), hot_categorical_columns),
        ('num', StandardScaler(), numeric_columns)
    ]
)

In [8]:
model = make_pipeline(
    preprocessor,
    DecisionTreeRegressor(max_depth=20, random_state=42)
)
model.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('cat', OrdinalEncoder(),
                                                  ['Place Code',
                                                   'Promotion Name',
                                                   'Store Kind',
                                                   'Is Recyclable?', 'Children',
                                                   'Degree', 'Work',
                                                   'Oreder Brand', 'Product',
                                                   'Department']),
                                                 ('ohe', OneHotEncoder(),
                                                  ['Marriage', 'Gender']),
                                                 ('num', StandardScaler(),
                                                  Index(['Store Sales', 'Store Cost', 'Store Area', 'Grocery Area',
       'Frozen Area', 'Meat Area', 'Gross Weight', 'N

In [9]:
# Check if the model fitted well or not
predictions = model.predict(X_train)

mse = mean_squared_error(y_train, predictions)
rmse = np.sqrt(mse)
rmse

16.24279568376655

In [10]:
predictions = model.predict(X_test)

mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
rmse

66.13992962598071

In [11]:
rfmodel = make_pipeline(
    preprocessor,
    RandomForestRegressor(n_estimators=100, random_state=42)
)
rfmodel.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('cat', OrdinalEncoder(),
                                                  ['Place Code',
                                                   'Promotion Name',
                                                   'Store Kind',
                                                   'Is Recyclable?', 'Children',
                                                   'Degree', 'Work',
                                                   'Oreder Brand', 'Product',
                                                   'Department']),
                                                 ('ohe', OneHotEncoder(),
                                                  ['Marriage', 'Gender']),
                                                 ('num', StandardScaler(),
                                                  Index(['Store Sales', 'Store Cost', 'Store Area', 'Grocery Area',
       'Frozen Area', 'Meat Area', 'Gross Weight', 'N

In [12]:
# Check if the model fitted well or not
predictions = rfmodel.predict(X_train)

mse = mean_squared_error(y_train, predictions)
rmse = np.sqrt(mse)
rmse

19.280206350134836

In [13]:
predictions = rfmodel.predict(X_test)

mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
rmse

53.03544924733378

In [14]:
def rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred)**2))

rmse_scorer = make_scorer(rmse, greater_is_better=False)

scores = cross_val_score(rfmodel, X_train, y_train, cv=5, scoring=rmse_scorer)

positive_scores = -scores

print("Cross-validation RMSE scores:", positive_scores)

Cross-validation RMSE scores: [53.65738053 56.10354203 52.34148074 52.18038972 50.51044931]


In [15]:
preprocessor.fit(X_train)

X_train_transformed = preprocessor.transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

## Try Lazypredict

In [35]:
reg = LazyRegressor(verbose=1,ignore_warnings=False, custom_metric=None)
models,predictions = reg.fit(X_train_transformed, X_test_transformed, y_train, y_test)

  2%|█▉                                                                                 | 1/42 [00:00<00:19,  2.13it/s]

{'Model': 'AdaBoostRegressor', 'R-Squared': 0.1229879692040099, 'Adjusted R-Squared': 0.11924272209740783, 'RMSE': 147.04110042057997, 'Time taken': 0.4701058864593506}


  7%|█████▉                                                                             | 3/42 [00:02<00:26,  1.45it/s]

{'Model': 'BaggingRegressor', 'R-Squared': 0.8722005560563311, 'Adjusted R-Squared': 0.8716547933063937, 'RMSE': 56.130699031326074, 'Time taken': 1.659067153930664}
{'Model': 'BayesianRidge', 'R-Squared': 0.016696374049479545, 'Adjusted R-Squared': 0.012497212657520018, 'RMSE': 155.69683667700198, 'Time taken': 0.11887073516845703}


 14%|███████████▊                                                                       | 6/42 [00:02<00:10,  3.58it/s]

...

{'Model': 'SGDRegressor', 'R-Squared': 0.013561621981421412, 'Adjusted R-Squared': 0.009349073747890069, 'RMSE': 155.9448183769587, 'Time taken': 0.1627511978149414}


 95%|██████████████████████████████████████████████████████████████████████████████    | 40/42 [22:30<00:19,  9.67s/it]

{'Model': 'SVR', 'R-Squared': 0.03079171715819351, 'Adjusted R-Squared': 0.02665274940228546, 'RMSE': 154.57687643348802, 'Time taken': 47.182443380355835}
{'Model': 'TransformedTargetRegressor', 'R-Squared': 0.016506461702418096, 'Adjusted R-Squared': 0.012306489296876832, 'RMSE': 155.7118713638147, 'Time taken': 0.04932141304016113}
{'Model': 'TweedieRegressor', 'R-Squared': 0.012382814440567103, 'Adjusted R-Squared': 0.008165232153480528, 'RMSE': 156.03796866653374, 'Time taken': 0.06249499320983887}
{'Model': 'XGBRegressor', 'R-Squared': 0.8527332424729093, 'Adjusted R-Squared': 0.8521043452877402, 'RMSE': 60.25433963077798, 'Time taken': 1.2772939205169678}
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001697 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2080
[LightGBM] [Info] Number of data points in the train set: 22580, n

100%|██████████████████████████████████████████████████████████████████████████████████| 42/42 [22:32<00:00, 32.20s/it]

{'Model': 'LGBMRegressor', 'R-Squared': 0.8287743411010376, 'Adjusted R-Squared': 0.8280431283228215, 'RMSE': 64.97112573940814, 'Time taken': 0.8558270931243896}


In [36]:
print(models)

                               Adjusted R-Squared  R-Squared   RMSE  \
Model                                                                 
ExtraTreesRegressor                          0.89       0.90  50.83   
RandomForestRegressor                        0.89       0.89  53.05   
BaggingRegressor                             0.87       0.87  56.13   
XGBRegressor                                 0.85       0.85  60.25   
LGBMRegressor                                0.83       0.83  64.97   
HistGradientBoostingRegressor                0.82       0.82  65.80   
DecisionTreeRegressor                        0.81       0.81  68.95   
ExtraTreeRegressor                           0.74       0.74  79.50   
GradientBoostingRegressor                    0.47       0.47 113.79   
AdaBoostRegressor                            0.12       0.12 147.04   
MLPRegressor                                 0.08       0.09 150.00   
SVR                                          0.03       0.03 154.58   
NuSVR 

In [25]:
from sklearn.ensemble import ExtraTreesRegressor

In [26]:
exmodel = make_pipeline(
    preprocessor,
    ExtraTreesRegressor(random_state=42)
)
exmodel.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('cat', OrdinalEncoder(),
                                                  ['Place Code',
                                                   'Promotion Name',
                                                   'Store Kind',
                                                   'Is Recyclable?', 'Children',
                                                   'Degree', 'Work',
                                                   'Oreder Brand', 'Product',
                                                   'Department']),
                                                 ('ohe', OneHotEncoder(),
                                                  ['Marriage', 'Gender']),
                                                 ('num', StandardScaler(),
                                                  Index(['Store Sales', 'Store Cost', 'Store Area', 'Grocery Area',
       'Frozen Area', 'Meat Area', 'Gross Weight', 'N

In [29]:
# Check if the model fitted well or not
predictions = exmodel.predict(X_train)

mse = mean_squared_error(y_train, predictions)
rmse = np.sqrt(mse)
rmse

8.697849117431592e-13

In [30]:
predictions = exmodel.predict(X_test)

mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
rmse

50.8250477268684

## Extract submissions

In [16]:
samples = pd.read_csv('data/sample_submission.csv')
samples.head()

,ID,Cost
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [17]:
test = pd.read_csv("data/cleaned_test.csv")
test.head()

,id,Place Code,Promotion Name,Store Kind,Store Sales,Store Cost,Is Recyclable?,Store Area,Grocery Area,Frozen Area,...,Department,Bar For Salad,Florist,Coffee Bar,Ready Food,Video Store,Gross Weight,Net Weight,Package Weight,Min. Person Yearly Income
0,0,B06ls_CA,Fantastic Discounts,Gourmet,11760000.00,4704000.00,yes,2201.06,1424.85,465.54,...,Snacks,1,1,1,1,1,31.83,28.78,3.05,50000.00
1,1,S01em_OR,Bag Stuffer,Deluxe,2160000.00,669600.00,no,2577.16,1735.17,505.07,...,Frozen Foods,1,1,1,1,1,29.94,27.04,2.91,70000.00
2,2,H11go_ZA,Pick Your Savings,Deluxe,1830000.00,823500.00,no,2837.58,2038.11,481.98,...,Dairy,1,1,1,1,1,29.22,26.31,2.91,130000.00
3,3,M10da_YU,Price Winners,Deluxe,8820000.00,4410000.00,no,2859.04,1871.16,593.93,...,Frozen Foods,1,1,1,1,1,28.05,25.00,3.05,10000.00
4,4,L05es_CA,Dollar Days,Supermarket,4320000.00,1987200.00,no,2193.97,1320.15,523.32,...,Beverages,0,1,0,0,0,23.55,20.64,2.91,30000.00


In [18]:
test.isna().sum()

id                           0
Place Code                   0
Promotion Name               0
Store Kind                   0
Store Sales                  0
Store Cost                   0
Is Recyclable?               0
Store Area                   0
Grocery Area                 0
Frozen Area                  0
Meat Area                    0
Marriage                     0
Gender                       0
Children                     0
Degree                       0
Work                         0
Oreder Brand                 0
Product                      0
Department                   0
Bar For Salad                0
Florist                      0
Coffee Bar                   0
Ready Food                   0
Video Store                  0
Gross Weight                 0
Net Weight                   0
Package Weight               0
Min. Person Yearly Income    0
dtype: int64

In [31]:
y_sub_pred = exmodel.predict(test)

In [32]:
samples["Cost"] = y_sub_pred

In [34]:
samples.to_csv("data/submission.csv", index=False)